In [1]:
from transformers import BertTokenizer, BertForSequenceClassification, BertModel, AutoTokenizer, AutoModel
from tqdm import tqdm

from dataset import NamesDataset

tokenizer = AutoTokenizer.from_pretrained('bert-base-uncased')

train_dict_path = 'data/train_dict.json'
val_dict_path = 'data/val_dict.json'

trainset = NamesDataset(train_dict_path, tokenizer=tokenizer, max_length=32)
valset = NamesDataset(val_dict_path, tokenizer=tokenizer, max_length=32)


trainset_labels = set(trainset.labels)
other_class_idx = 'other'
for idx, label in tqdm(enumerate(valset.labels)):
    if label not in trainset_labels:
        valset.labels[idx] = other_class_idx
valset.labels2target = trainset.labels2target
    

print('Train size:', len(trainset))
print('Val size:', len(valset))

/mnt/disk2/ndubrovnyi/words_similarity/.venv/lib/python3.8/site-packages/tqdm/auto.py:22: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm
3878it [00:00, 2875289.18it/s]

Train size: 14342
Val size: 3878


In [2]:
from model import NamesRecognition
from utils import fit_epoch, eval_epoch
from torch.utils.data import DataLoader

import torch
from torch import nn

from tqdm import tqdm

batch_size = 64
num_workers = 4

device = 'cuda:3'

margin = 0.5
scale = 15
num_classes = len(set(trainset.labels))
feature_extractor_name = 'bert-base-uncased'
embedding_size = 512
loss_func = 'adacos'

other_label = trainset.labels2target['other']

num_epochs = 30

train_loader = DataLoader(trainset, batch_size=batch_size, shuffle=True, num_workers=num_workers)
val_loader = DataLoader(valset, batch_size=batch_size, shuffle=False, num_workers=num_workers)

model = NamesRecognition(feature_extractor_name=feature_extractor_name, embedding_size=embedding_size,
                        margin=margin, scale=scale, num_of_classes=num_classes, loss_func=loss_func)

model.to(device)


criterion = nn.CrossEntropyLoss()
optimizer = torch.optim.Adam(model.parameters(), lr=3e-4)

scheduller = torch.optim.lr_scheduler.OneCycleLR(optimizer, max_lr=3e-4, epochs=num_epochs,
                                                 steps_per_epoch=len(train_loader), pct_start=0.1)

for param in model.parameters():
    param.requires_grad = False

for param in model.feature_extractor.encoder.layer[11].parameters():
    param.requires_grad = False
    
for param in model.feature_extractor.encoder.layer[10].parameters():
    param.requires_grad = False
    
for param in model.feature_extractor.encoder.layer[9].parameters():
    param.requires_grad = False
    
for param in model.feature_extractor.pooler.parameters():
    param.requires_grad = True
    
for param in model.head.parameters():
    param.requires_grad = True
    
for param in model.arc.parameters():
    param.requires_grad = True

for name, param in model.named_parameters():
    print(name, param.requires_grad)
    
    


version = 'BertAdaCos_v3'
    
best_acc = 0
layer_idx = 11
freq = 0
for epoch in range(1, num_epochs+1):
    
    train_loss = fit_epoch(model, train_loader, criterion, optimizer, device, scheduller)
    acc = eval_epoch(model, train_loader, val_loader, device, embedding_size, other_label)
    
    print('Epoch:', epoch)
    print('Train Loss:', train_loss)
    print('ACC@3:', acc)
    
    if acc > best_acc:
        torch.save(model, f'SaveModels/{version}_{epoch}.pth')
        best_acc = acc
        if freq >= 4:
            for param in model.feature_extractor.encoder.layer[layer_idx].parameters():
                param.requires_grad = True
            layer_idx -= 1
            freq = 0
            for name, param in model.named_parameters():
                print(name, param.requires_grad)
    elif (epoch > 3) and (freq >= 3):
        
        for param in model.feature_extractor.encoder.layer[layer_idx].parameters():
            param.requires_grad = True
        layer_idx -= 1
        freq = 0
        for name, param in model.named_parameters():
            print(name, param.requires_grad)
        
    with open(f'Logs/{version}.txt', 'a') as f:
        string = f'Epoch= {epoch} Train loss= {train_loss} ACC@3= {acc}\n'
        f.write(string)
        
    freq += 1

Some weights of the model checkpoint at bert-base-uncased were not used when initializing BertModel: ['cls.seq_relationship.weight', 'cls.seq_relationship.bias', 'cls.predictions.transform.dense.weight', 'cls.predictions.decoder.weight', 'cls.predictions.transform.dense.bias', 'cls.predictions.bias', 'cls.predictions.transform.LayerNorm.weight', 'cls.predictions.transform.LayerNorm.bias']
- This IS expected if you are initializing BertModel from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing BertModel from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).


feature_extractor.embeddings.word_embeddings.weight False
feature_extractor.embeddings.position_embeddings.weight False
feature_extractor.embeddings.token_type_embeddings.weight False
feature_extractor.embeddings.LayerNorm.weight False
feature_extractor.embeddings.LayerNorm.bias False
feature_extractor.encoder.layer.0.attention.self.query.weight False
feature_extractor.encoder.layer.0.attention.self.query.bias False
feature_extractor.encoder.layer.0.attention.self.key.weight False
feature_extractor.encoder.layer.0.attention.self.key.bias False
feature_extractor.encoder.layer.0.attention.self.value.weight False
feature_extractor.encoder.layer.0.attention.self.value.bias False
feature_extractor.encoder.layer.0.attention.output.dense.weight False
feature_extractor.encoder.layer.0.attention.output.dense.bias False
feature_extractor.encoder.layer.0.attention.output.LayerNorm.weight False
feature_extractor.encoder.layer.0.attention.output.LayerNorm.bias False
feature_extractor.encoder.layer.

4it [00:00,  9.31it/s]

Train loss: 9.712681770324707


25it [00:01, 21.95it/s]

...

Train loss: 0.7099583765760583


223it [00:20, 10.79it/s]

Train loss: 0.7101378788775449


225it [00:20, 10.76it/s]
100%|███████████████████████████████████████████| 61/61 [00:02<00:00, 22.88it/s]
/mnt/disk2/ndubrovnyi/words_similarity/.venv/lib/python3.8/site-packages/sklearn/neighbors/_nearest_centroid.py:168: UserWarning: Averaging for metrics other than euclidean and manhattan not supported. The average is set to be the mean.
  warnings.warn(


Epoch: 30
Train Loss: 0.7102160636583964
ACC@3: 0.9138731304796287
feature_extractor.embeddings.word_embeddings.weight False
feature_extractor.embeddings.position_embeddings.weight False
feature_extractor.embeddings.token_type_embeddings.weight False
feature_extractor.embeddings.LayerNorm.weight False
feature_extractor.embeddings.LayerNorm.bias False
feature_extractor.encoder.layer.0.attention.self.query.weight False
feature_extractor.encoder.layer.0.attention.self.query.bias False
feature_extractor.encoder.layer.0.attention.self.key.weight False
feature_extractor.encoder.layer.0.attention.self.key.bias False
feature_extractor.encoder.layer.0.attention.self.value.weight False
feature_extractor.encoder.layer.0.attention.self.value.bias False
feature_extractor.encoder.layer.0.attention.output.dense.weight False
feature_extractor.encoder.layer.0.attention.output.dense.bias False
feature_extractor.encoder.layer.0.attention.output.LayerNorm.weight False
feature_extractor.encoder.layer.0.att